In [2]:
import sys
print(sys.version)
sys.path.append('/Users/geba9152/LIET/liet/')

from liet_res_class import FitParse, fitparse_intersect

import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd


3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]


Date: 06/12/2024

This code is adapted from Jacob's code

I fixed the problematic PAS vs vs 5' UTR/last exon reference problem.

Here I am running an ANOVA in all perturbed and contorl meta samples to detect global changes in posterior values

# ANOVA data formatter function

In [3]:
def anova_data_formatter(param, fit_parses, treatments):
#     assert len(fit_parses) == len(treatments), "Number of treatments must match number of FitParse objects."
#     assert all(len(i.genes) == len(fit_parses[0].genes) for i in fit_parses), "FitParse objects have different genes."
    
    gene_ids = fit_parses[0].genes
    
    subject_array = [g for g in gene_ids for _ in range(len(treatments))]
    treatment_array = treatments * len(gene_ids)
    
    response_array = [fitp.__dict__[param] for fitp in fit_parses]
    response_array = [i for fitp_list in zip(*response_array) for i in fitp_list]
    
    # Check if the lengths of subject_array, treatment_array, and response_array are not equal
    if len(subject_array) != len(response_array) or len(treatment_array) != len(response_array):
        # Create a list of valid indices where the index is less than the length of response_array
        valid_indices = [i for i in range(len(subject_array)) if i < len(response_array)]
        
        # Filter subject_array to include only the elements at the valid indices
        subject_array = [subject_array[i] for i in valid_indices]
        
        # Filter treatment_array to include only the elements at the valid indices
        treatment_array = [treatment_array[i] for i in valid_indices]
        
        # Filter response_array to include only the elements at the valid indices
        response_array = [response_array[i] for i in valid_indices]

    
    # Debug statements to check lengths
    print(f'Length of subject_array (genes): {len(subject_array)}')
    print(f'Length of treatment_array: {len(treatment_array)}')
    print(f'Length of response_array (posterior val): {len(response_array)}')

    data = {
        'subject': subject_array,
        'treatment': treatment_array,
        param: response_array
    }
    
    df = pd.DataFrame(data)
    
    if param == 'mT':
        ann = pd.read_csv("/scratch/Users/geba9152/LIET-summer2024/meta-celltype-comparison/annotation/lietanns/chr1-6-3p-UTR.liet.ann", header=None, sep="\t")
        ann.columns = ['chr', 'start', 'PAS', 'subject', 'length', 'strand']
        ann = ann[['subject', 'length']]
        df_merged = df.merge(ann, on='subject')
        df_merged['mT'] = df_merged['mT'] - df_merged['length']
        df = df_merged[['subject', 'treatment', 'mT']]
    
    return df

# Ethan/Eric heat shock data
- 42 degrees C, 1 hour 

In [27]:
liet_res_dir = '/scratch/Shares/dowell/for_jacob/from_georgia/ethan-eric-hs-results-05142024/results/'

# er/et (Eric/Ethan), 1/2 (rep1/rep2), c/h (control/heat shock)
er1c = liet_res_dir+'Eric37_rep1.liet'
er2c = liet_res_dir+'Eric37_rep2.liet'
et1c = liet_res_dir+'Ethan37_rep1.liet'
et2c = liet_res_dir+'Ethan37_rep2.liet'
er1h = liet_res_dir+'Eric42_rep1.liet'
er2h = liet_res_dir+'Eric42_rep2.liet'
et1h = liet_res_dir+'Ethan42_rep1.liet'
et2h = liet_res_dir+'Ethan42_rep2.liet'

# d/t disomy (Eric) or Trisomic (Ethan)
d1c = FitParse(er1c, log_file=er1c+'.log')
d2c = FitParse(er2c, log_file=er2c+'.log')
t1c = FitParse(et1c, log_file=et1c+'.log')
t2c = FitParse(et2c, log_file=et2c+'.log')
d1h = FitParse(er1h, log_file=er1h+'.log')
d2h = FitParse(er2h, log_file=er2h+'.log')
t1h = FitParse(et1h, log_file=et1h+'.log')
t2h = FitParse(et2h, log_file=et2h+'.log')

In [28]:
liet_param = 'mT'
fit_parses = [d1c, d2c, t1c, t2c, d1h, d2h, t1h, t2h]
treatments = ['D', 'D', 'T', 'T', 'D+H', 'D+H', 'T+H', 'T+H']

anova_inputs = anova_data_formatter(liet_param, fit_parses, treatments)
anova_inputs['subject'] = anova_inputs['subject'].astype('category')

aovrm = AnovaRM(anova_inputs, liet_param, 'subject', within=['treatment'], aggregate_func='mean')

res = aovrm.fit()
print(res)

# Perform Tukey's HSD post-hoc test
tukey = pairwise_tukeyhsd(endog=anova_inputs[liet_param], groups=anova_inputs['treatment'], alpha=0.05)

# Display the results
print(tukey)

Length of subject_array (genes): 1304
Length of treatment_array: 1304
Length of response_array (posterior val): 1304
                 Anova
          F Value Num DF  Den DF  Pr > F
----------------------------------------
treatment 42.2969 3.0000 486.0000 0.0000



/Users/geba9152/miniconda3/envs/statsmodels/lib/python3.11/site-packages/statsmodels/stats/anova.py:509: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby([self.subject] + self.within,
/Users/geba9152/miniconda3/envs/statsmodels/lib/python3.11/site-packages/statsmodels/stats/anova.py:511: FutureWarning: The provided callable <function mean at 0x2ae0d3c4fd80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg(self.aggregate_func))


    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff  p-adj    lower      upper   reject
-----------------------------------------------------------
     D    D+H  1996.4229 0.0001   788.6065 3204.2392   True
     D      T   -92.6669 0.9973 -1300.4832 1115.1495  False
     D    T+H  1988.6583 0.0001   780.8419 3196.4746   True
   D+H      T -2089.0897 0.0001 -3296.9061 -881.2734   True
   D+H    T+H    -7.7646    1.0 -1215.5809 1200.0518  False
     T    T+H  2081.3252 0.0001   873.5088 3289.1415   True
-----------------------------------------------------------


# CDK7i + Heat Shock OV90

- All drug treated data: 30 min pre-treatment prior to any heat shock (42 degrees C)
- Concentration of SY5609 = 50 nM

In [51]:
meta = pd.read_csv('/scratch/Shares/dowell/ov90_cdk7i/PROseq/ov90_cdk7i_metadata_PRO.txt', sep = "\t")
meta.columns
# meta['drug_pretreatment']

Index(['assay', 'sample_read1', 'sample_read2', 'id', 'idx', 'coded_reps',
       'replicate', 'batch', 'advanced_batch', 'genotype', 'drug_pretreatment',
       'hs_time', 'recovery_time', 'total_time', 'tx_type', 'group',
       'millionsmapped', 'spikein', 'spikein_mm', 'VSI', 'VSI_v1', 'linear_sf',
       'deseq_sf', 'dm6_VS', 'dm6_VS_v1', 'dm6_linear_sf', 'dm6_deseq2_sf',
       'ercc_ctrlgenes_deseq2_sf', 'fastq_path', 'analysis_path', 'path_sep',
       'color', 'millionsmapped_updated', 'deseq2_ercc_norm'],
      dtype='object')

In [29]:
liet_res_dir = '/scratch/Users/geba9152/LIET-summer2024/cdk7i-ov90/results/tight-sT/'

# liet paths
dmso_0_1_path = liet_res_dir+'DMSO_0_R1.liet'
dmso_0_2_path = liet_res_dir+'DMSO_0_R2.liet'
dmso_20_1_path = liet_res_dir+'DMSO_20_R1.liet'
dmso_20_2_path = liet_res_dir+'DMSO_20_R2.liet'
dmso_45_1_path = liet_res_dir+'DMSO_45_R1.liet'
dmso_45_2_path = liet_res_dir+'DMSO_45_R2.liet'

sy_0_1_path = liet_res_dir+'SY_0_R1.liet'
sy_0_2_path = liet_res_dir+'SY_0_R2.liet'
sy_20_1_path = liet_res_dir+'SY_20_R1.liet'
sy_20_2_path = liet_res_dir+'SY_20_R2.liet'
sy_45_1_path = liet_res_dir+'SY_45_R1.liet'
sy_45_2_path = liet_res_dir+'SY_45_R2.liet'


# fp objects
dmso_0_1 = FitParse(dmso_0_1_path, log_file=dmso_0_1_path+'.log')
dmso_0_2 = FitParse(dmso_0_2_path, log_file=dmso_0_2_path+'.log')
dmso_20_1 = FitParse(dmso_20_1_path, log_file=dmso_20_1_path+'.log')
dmso_20_2 = FitParse(dmso_20_2_path, log_file=dmso_20_2_path+'.log')
dmso_45_1 = FitParse(dmso_45_1_path, log_file=dmso_45_1_path+'.log')
dmso_45_2 = FitParse(dmso_45_2_path, log_file=dmso_45_2_path+'.log')

sy_0_1 = FitParse(sy_0_1_path, log_file=sy_0_1_path+'.log')
sy_0_2 = FitParse(sy_0_2_path, log_file=sy_0_2_path+'.log')
sy_20_1 = FitParse(sy_20_1_path, log_file=sy_20_1_path+'.log')
sy_20_2 = FitParse(sy_20_2_path, log_file=sy_20_2_path+'.log')
sy_45_1 = FitParse(sy_45_1_path, log_file=sy_45_1_path+'.log')
sy_45_2 = FitParse(sy_45_2_path, log_file=sy_45_2_path+'.log')

In [30]:
liet_param = 'sT'

fit_parses = [dmso_0_1, dmso_0_2, dmso_20_1, dmso_20_2, dmso_45_1, dmso_45_2, sy_0_1, sy_0_2, sy_20_1, sy_20_2, sy_45_1, sy_45_2]
treatments = ['DMSO_0', 'DMSO_0','DMSO_20', 'DMSO_20','DMSO_45','DMSO_45','SY_0', 'SY_0','SY_20', 'SY_20','SY_45','SY_45']

anova_inputs = anova_data_formatter(liet_param, fit_parses, treatments)
anova_inputs['subject'] = anova_inputs['subject'].astype('category')

aovrm = AnovaRM(anova_inputs, liet_param, 'subject', within=['treatment'], aggregate_func='mean')

res = aovrm.fit()
print(res)

# Perform Tukey's HSD post-hoc test
tukey = pairwise_tukeyhsd(endog=anova_inputs[liet_param], groups=anova_inputs['treatment'], alpha=0.05)

# Display the results
print(tukey)

Length of subject_array (genes): 1956
Length of treatment_array: 1956
Length of response_array (posterior val): 1956
                 Anova
          F Value Num DF  Den DF  Pr > F
----------------------------------------
treatment 15.8938 5.0000 810.0000 0.0000



/Users/geba9152/miniconda3/envs/statsmodels/lib/python3.11/site-packages/statsmodels/stats/anova.py:509: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby([self.subject] + self.within,
/Users/geba9152/miniconda3/envs/statsmodels/lib/python3.11/site-packages/statsmodels/stats/anova.py:511: FutureWarning: The provided callable <function mean at 0x2ae0d3c4fd80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg(self.aggregate_func))


    Multiple Comparison of Means - Tukey HSD, FWER=0.05     
 group1  group2  meandiff p-adj    lower      upper   reject
------------------------------------------------------------
 DMSO_0 DMSO_20  825.2788 0.0002   283.2613 1367.2963   True
 DMSO_0 DMSO_45 1100.7294    0.0    558.712 1642.7469   True
 DMSO_0    SY_0  122.3052 0.9877  -419.7123  664.3227  False
 DMSO_0   SY_20   910.751    0.0   368.7336 1452.7685   True
 DMSO_0   SY_45  819.9923 0.0002   277.9748 1362.0098   True
DMSO_20 DMSO_45  275.4506 0.6964  -266.5668  817.4681  False
DMSO_20    SY_0 -702.9736  0.003 -1244.9911 -160.9562   True
DMSO_20   SY_20   85.4722 0.9977  -456.5452  627.4897  False
DMSO_20   SY_45   -5.2865    1.0   -547.304   536.731  False
DMSO_45    SY_0 -978.4243    0.0 -1520.4417 -436.4068   True
DMSO_45   SY_20 -189.9784 0.9181  -731.9959  352.0391  False
DMSO_45   SY_45 -280.7371 0.6788  -822.7546  261.2803  False
   SY_0   SY_20  788.4459 0.0005   246.4284 1330.4633   True
   SY_0   SY_45  697.687

# CDK7i SY5609 HCT

`#TODO: run LIET on the rest of this data`

- 50nM SY5609
- 100nM KB0742
- IFN gamma was used at 10ng/mL

Combo scheme is as follows: time 0min—cells are treated with DMSO or inhibitor/s, time 30min—IFN gamma is added, time 75min—cells are harvested.

In [25]:
liet_res_dir = '/scratch/Users/geba9152/LIET-summer2024/cdk7i-hct/results/tight-sT/'

# liet paths
c1_path = liet_res_dir+'DMSO_1.liet'
c2_path = liet_res_dir+'DMSO_2.liet'
sy5609_1_path = liet_res_dir+'SY5609_1.liet'
sy5609_2_path = liet_res_dir+'SY5609_2.liet'

# fp objects
c1 = FitParse(c1_path, log_file=c1_path+'.log')
c2 = FitParse(c2_path, log_file=c2_path+'.log')
s1 = FitParse(sy5609_1_path, log_file=sy5609_1_path+'.log')
s2 = FitParse(sy5609_2_path, log_file=sy5609_2_path+'.log')

In [26]:
liet_param = 'mT'

fit_parses = [c1, c2, s1, s2]
treatments = ['C', 'C','CDK7i-noncov', 'CDK7i-noncov']

anova_inputs = anova_data_formatter(liet_param, fit_parses, treatments)
anova_inputs['subject'] = anova_inputs['subject'].astype('category')

aovrm = AnovaRM(anova_inputs, liet_param, 'subject', within=['treatment'], aggregate_func='mean')

res = aovrm.fit()
print(res)

# Perform Tukey's HSD post-hoc test
tukey = pairwise_tukeyhsd(endog=anova_inputs[liet_param], groups=anova_inputs['treatment'], alpha=0.05)

# Display the results
print(tukey)

Length of subject_array (genes): 652
Length of treatment_array: 652
Length of response_array (posterior val): 652
                 Anova
          F Value Num DF  Den DF  Pr > F
----------------------------------------
treatment 60.1285 1.0000 162.0000 0.0000



/Users/geba9152/miniconda3/envs/statsmodels/lib/python3.11/site-packages/statsmodels/stats/anova.py:509: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby([self.subject] + self.within,
/Users/geba9152/miniconda3/envs/statsmodels/lib/python3.11/site-packages/statsmodels/stats/anova.py:511: FutureWarning: The provided callable <function mean at 0x2ae0d3c4fd80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg(self.aggregate_func))


     Multiple Comparison of Means - Tukey HSD, FWER=0.05      
group1    group2     meandiff p-adj   lower     upper   reject
--------------------------------------------------------------
     C CDK7i-noncov 2842.8713   0.0 1478.5224 4207.2202   True
--------------------------------------------------------------


# IFN/CDK7i (SY351)/CDK9i HCT
- 50nM SY351
- 100nM KB0742 
- IFN gamma was used at 10ng/mL

Combo scheme is as follows: time 0min—cells are treated with DMSO or inhibitor/s, time 30min—IFN gamma is added, time 75min—cells are harvested.

In [22]:
liet_res_dir = '/scratch/Users/geba9152/LIET-summer2024/KB0742-SY351-batch/results/tight-sT/'

# liet paths
c1_path = liet_res_dir+'DMSO_1_S1_L001_R1_001.liet'
c2_path = liet_res_dir+'DMSO_2_S6_L001_R1_001.liet'
ifn1_path = liet_res_dir+'IFN_1.liet'
ifn2_path = liet_res_dir+'IFN_2.liet'
cdk91_path = liet_res_dir+'KB0742_1_S4_L001_R1_001.liet'
cdk92_path = liet_res_dir+'KB0742_2_S9_L001_R1_001.liet'
cdk71_path = liet_res_dir+'SY351_1_S3_L001_R1_001.liet'
cdk72_path = liet_res_dir+'SY351_2_S8_L001_R1_001.liet'
cdk971_path = liet_res_dir+'SY-KB_1_S1_L001_R1_001.liet'
cdk972_path = liet_res_dir+'SY-KB_1_S5_L001_R1_001.liet'
cdk973_path = liet_res_dir+'SY-KB_2_S10_L001_R1_001.liet'

# fp objects
c1 = FitParse(c1_path, log_file=c1_path+'.log')
c2 = FitParse(c2_path, log_file=c2_path+'.log')
ifn1 = FitParse(ifn1_path, log_file=ifn1_path+'.log')
ifn2 = FitParse(ifn2_path, log_file=ifn2_path+'.log')
cdk91 = FitParse(cdk91_path, log_file=cdk91_path+'.log')
cdk92 = FitParse(cdk92_path, log_file=cdk92_path+'.log')
cdk71 = FitParse(cdk71_path, log_file=cdk71_path+'.log')
cdk72 = FitParse(cdk72_path, log_file=cdk72_path+'.log')
cdk971 = FitParse(cdk971_path, log_file=cdk971_path+'.log')
cdk972 = FitParse(cdk972_path, log_file=cdk972_path+'.log')
cdk973 = FitParse(cdk973_path, log_file=cdk973_path+'.log')

In [24]:
liet_param = 'mT'
fit_parses = [c1, c2, ifn1, ifn2, cdk91, cdk92, cdk71, cdk72, cdk971, cdk972, cdk973]
treatments = ['C', 'C', 'IFN', 'IFN', 'CDK9', 'CDK9', 'CDK7', 'CDK7', 'CDK7+9', 'CDK7+9', 'CDK7+9']

# fit_parses = [c1, c2, ifn1, ifn2, cdk91, cdk92, cdk71, cdk72, cdk971, cdk972]
# treatments = ['C', 'C', 'IFN', 'IFN', 'CDK9', 'CDK9', 'CDK7', 'CDK7', 'CDK7+9', 'CDK7+9']


anova_inputs = anova_data_formatter(liet_param, fit_parses, treatments)
anova_inputs['subject'] = anova_inputs['subject'].astype('category')

aovrm = AnovaRM(anova_inputs, liet_param, 'subject', within=['treatment'], aggregate_func='mean')

res = aovrm.fit()
print(res)

# Perform Tukey's HSD post-hoc test
tukey = pairwise_tukeyhsd(endog=anova_inputs[liet_param], groups=anova_inputs['treatment'], alpha=0.05)

# Display the results
print(tukey)

Length of subject_array (genes): 1749
Length of treatment_array: 1749
Length of response_array (posterior val): 1749
                 Anova
          F Value Num DF  Den DF  Pr > F
----------------------------------------
treatment  0.2867 4.0000 632.0000 0.8866



/Users/geba9152/miniconda3/envs/statsmodels/lib/python3.11/site-packages/statsmodels/stats/anova.py:509: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby([self.subject] + self.within,
/Users/geba9152/miniconda3/envs/statsmodels/lib/python3.11/site-packages/statsmodels/stats/anova.py:511: FutureWarning: The provided callable <function mean at 0x2ae0d3c4fd80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg(self.aggregate_func))


    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff  p-adj    lower      upper   reject
-----------------------------------------------------------
     C   CDK7  -135.5285    1.0 -8340.0746 8069.0176  False
     C CDK7+9 -2241.6192 0.9254 -9731.3108 5248.0725  False
     C   CDK9 -1125.4688 0.9958  -9330.015 7079.0773  False
     C    IFN  -371.4239 0.9999   -8575.97 7833.1223  False
  CDK7 CDK7+9 -2106.0907 0.9398 -9595.7823  5383.601  False
  CDK7   CDK9  -989.9403 0.9975 -9194.4865 7214.6058  False
  CDK7    IFN  -235.8954    1.0 -8440.4415 7968.6508  False
CDK7+9   CDK9  1116.1503 0.9942 -6373.5413  8605.842  False
CDK7+9    IFN  1870.1953 0.9604 -5619.4964 9359.8869  False
  CDK9    IFN    754.045 0.9991 -7450.5012 8958.5911  False
-----------------------------------------------------------


# Meta cross cell type comparison

In [15]:
liet_res_dir = '/scratch/Users/geba9152/LIET-summer2024/meta-celltype-comparison/results/tight-sT/'

# liet paths
meta_OCILY1 = liet_res_dir+'meta_OCILY1.liet'
meta_G401 = liet_res_dir+'meta_G401.liet'
meta_HCT116 = liet_res_dir+'meta_HCT116.liet'
meta_MEL624 = liet_res_dir+'meta_MEL624.liet'
meta_HeLa = liet_res_dir+'meta_HeLa.liet'
meta_THP1 = liet_res_dir+'meta_THP1.liet'
meta_foreskin_fibroblast = liet_res_dir+'meta_foreskin_fibroblast.liet'
meta_A375 = liet_res_dir+'meta_A375.liet'
meta_SUDHL4 = liet_res_dir+'meta_SUDHL4.liet'
meta_lymphoblast = liet_res_dir+'meta_lymphoblast.liet'
meta_Jurkat_T_cell = liet_res_dir+'meta_Jurkat_T_cell.liet'
meta_ESC = liet_res_dir+'meta_ESC.liet'
meta_LC2ad = liet_res_dir+'meta_LC2ad.liet'
meta_Ramos = liet_res_dir+'meta_Ramos.liet'
meta_HEK239T_HEK239 = liet_res_dir+'meta_HEK239T-HEK239.liet'
meta_BEAS2B = liet_res_dir+'meta_BEAS2B.liet'
meta_NUDUL1 = liet_res_dir+'meta_NUDUL1.liet'
meta_K562 = liet_res_dir+'meta_K562.liet'
meta_MCF7 = liet_res_dir+'meta_MCF7.liet'
meta_Kasumi1 = liet_res_dir+'meta_Kasumi1.liet'
meta_S2VP10 = liet_res_dir+'meta_S2VP10.liet'
meta_KBM7 = liet_res_dir+'meta_KBM7.liet'
meta_CD34_erythroblast = liet_res_dir+'meta_CD34_erythroblast.liet'
meta_HAP1 = liet_res_dir+'meta_HAP1.liet'
meta_U936 = liet_res_dir+'meta_U936.liet'
meta_CD4_T_cell = liet_res_dir+'meta_CD4_T_cell.liet'
meta_A549 = liet_res_dir+'meta_A549.liet'

# fp objects
meta_OCILY1_fp = FitParse(meta_OCILY1, log_file=meta_OCILY1+'.log')
meta_G401_fp = FitParse(meta_G401, log_file=meta_G401+'.log')
meta_HCT116_fp = FitParse(meta_HCT116, log_file=meta_HCT116+'.log')
meta_MEL624_fp = FitParse(meta_MEL624, log_file=meta_MEL624+'.log')
meta_HeLa_fp = FitParse(meta_HeLa, log_file=meta_HeLa+'.log')
meta_THP1_fp = FitParse(meta_THP1, log_file=meta_THP1+'.log')
meta_foreskin_fibroblast_fp = FitParse(meta_foreskin_fibroblast, log_file=meta_foreskin_fibroblast+'.log')
meta_A375_fp = FitParse(meta_A375, log_file=meta_A375+'.log')
meta_SUDHL4_fp = FitParse(meta_SUDHL4, log_file=meta_SUDHL4+'.log')
meta_lymphoblast_fp = FitParse(meta_lymphoblast, log_file=meta_lymphoblast+'.log')
meta_Jurkat_T_cell_fp = FitParse(meta_Jurkat_T_cell, log_file=meta_Jurkat_T_cell+'.log')
meta_ESC_fp = FitParse(meta_ESC, log_file=meta_ESC+'.log')
meta_LC2ad_fp = FitParse(meta_LC2ad, log_file=meta_LC2ad+'.log')
meta_Ramos_fp = FitParse(meta_Ramos, log_file=meta_Ramos+'.log')
meta_HEK239T_HEK239_fp = FitParse(meta_HEK239T_HEK239, log_file=meta_HEK239T_HEK239+'.log')
meta_BEAS2B_fp = FitParse(meta_BEAS2B, log_file=meta_BEAS2B+'.log')
meta_NUDUL1_fp = FitParse(meta_NUDUL1, log_file=meta_NUDUL1+'.log')
meta_K562_fp = FitParse(meta_K562, log_file=meta_K562+'.log')
meta_MCF7_fp = FitParse(meta_MCF7, log_file=meta_MCF7+'.log')
meta_Kasumi1_fp = FitParse(meta_Kasumi1, log_file=meta_Kasumi1+'.log')
meta_S2VP10_fp = FitParse(meta_S2VP10, log_file=meta_S2VP10+'.log')
meta_KBM7_fp = FitParse(meta_KBM7, log_file=meta_KBM7+'.log')
meta_CD34_erythroblast_fp = FitParse(meta_CD34_erythroblast, log_file=meta_CD34_erythroblast+'.log')
meta_HAP1_fp = FitParse(meta_HAP1, log_file=meta_HAP1+'.log')
meta_U936_fp = FitParse(meta_U936, log_file=meta_U936+'.log')
meta_CD4_T_cell_fp = FitParse(meta_CD4_T_cell, log_file=meta_CD4_T_cell+'.log')
meta_A549_fp = FitParse(meta_A549, log_file=meta_A549+'.log')


CHECK LINE: ['NUCKS1|NM_022731.5: fitting error']
CHECK LINE: ['MAN1A2|NM_006699.5: fitting error']
CHECK LINE: ['MAN1A2|NM_006699.5: fitting error']
CHECK LINE: ['NUCKS1|NM_022731.5: fitting error']
CHECK LINE: ['RPL37A|NM_000998.5: fitting error']


In [16]:
fit_parses = [
    meta_OCILY1_fp,
    meta_G401_fp,
    meta_HCT116_fp,
    meta_MEL624_fp,
    meta_HeLa_fp,
    meta_THP1_fp,
    meta_foreskin_fibroblast_fp,
    meta_A375_fp,
    meta_SUDHL4_fp,
    meta_lymphoblast_fp,
    meta_Jurkat_T_cell_fp,
    meta_ESC_fp,
    meta_LC2ad_fp,
    meta_Ramos_fp,
    meta_HEK239T_HEK239_fp,
    meta_BEAS2B_fp,
    meta_NUDUL1_fp,
    meta_K562_fp,
    meta_MCF7_fp,
    meta_Kasumi1_fp,
    meta_S2VP10_fp,
    meta_KBM7_fp,
    meta_CD34_erythroblast_fp,
    meta_HAP1_fp,
    meta_U936_fp,
    meta_CD4_T_cell_fp,
    meta_A549_fp
]

treatments = [
    'meta_OCILY1',
    'meta_G401',
    'meta_HCT116',
    'meta_MEL624',
    'meta_HeLa',
    'meta_THP1',
    'meta_foreskin_fibroblast',
    'meta_A375',
    'meta_SUDHL4',
    'meta_lymphoblast',
    'meta_Jurkat_T_cell',
    'meta_ESC',
    'meta_LC2ad',
    'meta_Ramos',
    'meta_HEK239T_HEK239',
    'meta_BEAS2B',
    'meta_NUDUL1',
    'meta_K562',
    'meta_MCF7',
    'meta_Kasumi1',
    'meta_S2VP10',
    'meta_KBM7',
    'meta_CD34_erythroblast',
    'meta_HAP1',
    'meta_U936',
    'meta_CD4_T_cell',
    'meta_A549'
]

print(len(treatments))

liet_param = 'sT'

anova_inputs = anova_data_formatter(liet_param, fit_parses, treatments)
anova_inputs['subject'] = anova_inputs['subject'].astype('category')

aovrm = AnovaRM(anova_inputs, liet_param, 'subject', within=['treatment'], aggregate_func='mean')

res = aovrm.fit()
print(res)

# Perform Tukey's HSD post-hoc test
tukey = pairwise_tukeyhsd(endog=anova_inputs[liet_param], groups=anova_inputs['treatment'], alpha=0.05)

# Display the results
print(tukey)


27
Length of subject_array (genes): 4347
Length of treatment_array: 4347
Length of response_array (posterior val): 4347
                  Anova
          F Value  Num DF   Den DF  Pr > F
------------------------------------------
treatment  4.0602 26.0000 4160.0000 0.0000

                      Multiple Comparison of Means - Tukey HSD, FWER=0.05                      
         group1                   group2           meandiff  p-adj    lower      upper   reject
-----------------------------------------------------------------------------------------------
               meta_A375                meta_A549  -345.8922    1.0 -1578.4755   886.691  False
               meta_A375              meta_BEAS2B  -330.4082    1.0 -1562.9914   902.175  False
               meta_A375   meta_CD34_erythroblast  -300.6593    1.0 -1533.2425  931.9239  False
               meta_A375          meta_CD4_T_cell  -336.5102    1.0 -1569.0934   896.073  False
               meta_A375                 meta_ESC  -20